In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://almaty.hh.kz/search/vacancy?text=python&area=160'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://almaty.hh.kz' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'almaty.hh.kz',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [5]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160
Cтраница №2, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160&page=3&hhtmFrom=vacancy_search_list
Парсинг завершен


In [6]:
len(result)

80

In [7]:
result

[{'vacancy_name': 'Разработчик',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://almaty.hh.kz/vacancy/74417356?from=vacancy_search_list&query=python',
  'vacancy_source': 'almaty.hh.kz'},
 {'vacancy_name': 'Backend Developer (Python) программист-разработчик',
  'vacancy_salary': 'от 200\u202f000 KZT',
  'min_salary': 200000,
  'max_salary': None,
  'currency_salary': 'KZT',
  'vacancy_link': 'https://almaty.hh.kz/vacancy/74147867?from=vacancy_search_list&query=python',
  'vacancy_source': 'almaty.hh.kz'},
 {'vacancy_name': 'Python Developer (Middle/Senior)',
  'vacancy_salary': '2\u202f000 – 4\u202f000 USD',
  'min_salary': 2000,
  'max_salary': 4000,
  'currency_salary': 'USD',
  'vacancy_link': 'https://almaty.hh.kz/vacancy/73583074?from=vacancy_search_list&query=python',
  'vacancy_source': 'almaty.hh.kz'},
 {'vacancy_name': 'Python Engineer, Remote US timezone',
  'vacancy_salary': 'до 4\u202f000 USD',
  'm